<h1>OFD1: Simpson-Golabi-Behmel syndrome, type 2. Budny et al, 2006</h1>
<p>Data extracted from <a href="https://pubmed.ncbi.nlm.nih.gov/16783569/" target="__blank">Budny, et al. (2006) A novel X-linked recessive mental retardation syndrome comprising macrocephaly and ciliary dysfunction is allelic to oral-facial-digital type I syndrome</a>.</p>

In [1]:
from pyphetools.creation import TemplateImporter

In [2]:
from pyphetools.creation import TemplateImporter
from pyphetools.visualization import IndividualTable, QcVisualizer
from IPython.display import display, HTML
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.66


In [3]:
template = "input/OFD1_SGBS2_individuals.xlsx"
hp_json = "../hp.json"
created_by = "0000-0002-0736-9199"

In [5]:
timporter = TemplateImporter(template=template, hp_json=hp_json, created_by=created_by)
# X-linked recessive, set hemizygous argument to True
individual_list, cvalidator = timporter.import_phenopackets_from_template(hemizygous=True)

HPO version 2024-03-06
Created encoders for 30 fields
 we got OMIM:300209  NM_003611.3
We output 3 GA4GH phenopackets to the directory phenopackets


In [6]:
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

In [7]:
table = IndividualTable(cvalidator.get_error_free_individual_list())
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
V: 1 (MALE; P11Y),"Simpson-Golabi-Behmel syndrome, type 2 (OMIM:300209)",NM_003611.3:c.2122_2125dup (hemizygous),"Macrocephaly (HP:0000256); Recurrent upper respiratory tract infections (HP:0002788); Recurrent lower respiratory tract infections (HP:0002783); Intellectual disability, severe (HP:0010864); Global developmental delay (HP:0001263); High palate (HP:0000218); Low-set ears (HP:0000369); Broad thumb (HP:0011304); Short finger (HP:0009381); Obesity (HP:0001513); excluded: Hypotonia (HP:0001252); excluded: Postaxial hand polydactyly (HP:0001162); excluded: Inguinal hernia (HP:0000023)"
IV: 3 (MALE; P1Y6M),"Simpson-Golabi-Behmel syndrome, type 2 (OMIM:300209)",NM_003611.3:c.2122_2125dup (hemizygous),Postaxial hand polydactyly (HP:0001162); Macrocephaly (HP:0000256); Recurrent upper respiratory tract infections (HP:0002788); Recurrent lower respiratory tract infections (HP:0002783); Global developmental delay (HP:0001263); excluded: Hypotonia (HP:0001252); excluded: High palate (HP:0000218); excluded: Low-set ears (HP:0000369); excluded: Broad thumb (HP:0011304); excluded: Short finger (HP:0009381); excluded: Obesity (HP:0001513); excluded: Inguinal hernia (HP:0000023)
IV: 11 (MALE; P3Y),"Simpson-Golabi-Behmel syndrome, type 2 (OMIM:300209)",NM_003611.3:c.2122_2125dup (hemizygous),Hypotonia (HP:0001252); Global developmental delay (HP:0001263); Short finger (HP:0009381); Inguinal hernia (HP:0000023); excluded: Recurrent upper respiratory tract infections (HP:0002788); excluded: Recurrent lower respiratory tract infections (HP:0002783); excluded: High palate (HP:0000218); excluded: Low-set ears (HP:0000369); excluded: Broad thumb (HP:0011304); excluded: Obesity (HP:0001513); excluded: Postaxial hand polydactyly (HP:0001162)
